In [1]:
'''
E91
'''

'\nE91\n'

In [2]:
from qiskit import BasicAer,execute,QuantumCircuit,QuantumRegister,ClassicalRegister
import matplotlib.pyplot as plt
from numpy.random import randint,random_integers
import numpy as np
import math
backend = BasicAer.get_backend('dm_simulator')

In [3]:
options_without_noise={
    'plot': False,
}
options_noise = { 
    'plot': False,
"rotation_error": {'rx':[1.0, 0.0], 'ry':[1.0, 0.0], 'rz':[1.0,0.0]},
"tsp_model_error": [1.0, 0.0],
"thermal_factor": 1.0,
"decoherence_factor": 1.0,
"depolarization_factor": 1.0,
"bell_depolarization_factor": 1.0,
"decay_factor": 1.0,
}


In [4]:
envs = input(print("Enter either noise or noiseless in all lowercase"))
trace_pass = input(print("Enter yes or no in all lowercase"))

Enter either noise or noiseless in all lowercase


None noiseless


Enter yes or no in all lowercase


None no


In [6]:
if(envs == "noisy"):
    print("""Enter the noise_model""")
    print("Enter values for rotation error:")
    while(1):
        try:
            options_noise["rotation_error"]["rx"] = [float(x) for x in input("\n\nEnter 2 values, for rx(space separated):").split()]
            if( options_noise["rotation_error"]["rx"][0] + options_noise["rotation_error"]["rx"][1] > 1 ):
                raise AttributeError
        except AttributeError:
            print("Summation of the values for rx cannot be greater than 1.\n Try again with valid values...")
            continue
        except ValueError:
            print("Please enter integer or float values only, in correct format...\nTry again, entering the values.")
            continue
        else:
            break
    
    while(1):
        try:
            options_noise["rotation_error"]["ry"] = [float(x) for x in input("Enter 2 values, for ry(space separated):").split()]
            if( options_noise["rotation_error"]["ry"][0] + options_noise["rotation_error"]["ry"][1] > 1 ):
                raise AttributeError
        except AttributeError:
            print("Summation of the values for rx cannot be greater than 1.\n Try again with valid values...")
            continue
        except ValueError:
            print("Please enter integer or float values only, in correct format...\nTry again, entering the values.")
            continue
        else:
            break
    
    while(1):
        try:
            options_noise["rotation_error"]["rz"] = [float(x) for x in input("Enter 2 values, for rz(space separated):").split()]
            if( options_noise["rotation_error"]["rz"][0] + options_noise["rotation_error"]["rz"][1] > 1 ):
                raise AttributeError
        except AttributeError:
            print("Summation of the values for rx cannot be greater than 1.\n Try again with valid values...")
            continue
        except ValueError:
            print("Please enter integer or float values only, in correct format...\nTry again, entering the values.")
            continue
        else:
            break
            
    # -----------------------------------------------------------------------------------------------------------------------------------------
            
    while(1):
        try:
            options_noise["tsp_model_error"] = [float(x) for x in input("Enter 2 float values, for tsp_model_error(space separated):").split()]
            if( options_noise["tsp_model_error"][0] + options_noise["tsp_model_error"][1] > 1 ):
                raise AttributeError
        except AttributeError:
            print("Summation of the values for tsp_model_error cannot be greater than 1.\n Try again with valid values...")
            continue
        except ValueError:
            print("Please enter integer or float values only, in correct format...\nTry again, entering the values.")
            continue
        else:
            break
    
            
    # -----------------------------------------------------------------------------------------------------------------------------------------
    
    while(1):
        try:
            options_noise["thermal_factor"] = float( input("Enter a float value(less than equa to 1), for THERMAL FACTOR:" ) )
            if( options_noise["thermal_factor"] > 1 ):
                raise AttributeError
        except AttributeError:
            print("THERMAL FACTOR value cannot be greater than 1.\n Try again with valid values...")
            continue
        except ValueError:
            print("Please enter integer or float values only, in correct format...\nTry again, entering the values.")
            continue
        else:
            break
    
    # -----------------------------------------------------------------------------------------------------------------------------------------
    
    while(1):
        try:
            options_noise["decoherence_factor"] = float( input("Enter a float value(less than equal to 1), for DECOHERENCE FACTOR:" ) )
            if( options_noise["decoherence_factor"] > 1 ):
                raise AttributeError
        except AttributeError:
            print("DECOHERENCE FACTOR value cannot be greater than 1.\n Try again with valid values...")
            continue
        except ValueError:
            print("Please enter integer or float values only, in correct format...\nTry again, entering the values.")
            continue
        else:
            break
    
    # -----------------------------------------------------------------------------------------------------------------------------------------
    
    while(1):
        try:
            options_noise["depolarization_factor"] = float( input("Enter a float value(less than equa to 1), for DEPOLARIZATION FACTOR:" ) )
            if( options_noise["depolarization_factor"] > 1 ):
                raise AttributeError
        except AttributeError:
            print("DEPOLARIZATION FACTOR value cannot be greater than 1.\n Try again with valid values...")
            continue
        except ValueError:
            print("Please enter integer or float values only, in correct format...\nTry again, entering the values.")
            continue
        else:
            break
                                                    
    # -----------------------------------------------------------------------------------------------------------------------------------------
    
    while(1):
        try:
            options_noise["bell_depolarization_factor"] = float( input("Enter a float value(less than equal to 1), for BELL DEPOLARIZATION FACTOR:" ) )
            if( options_noise["bell_depolarization_factor"] > 1 ):
                raise AttributeError
        except AttributeError:
            print("BELL DEPOLARIZATION FACTOR value cannot be greater than 1.\n Try again with valid values...")
            continue
        except ValueError:
            print("Please enter integer or float values only, in correct format...\nTry again, entering the values.")
            continue
        else:
            break
                                                    
    # -----------------------------------------------------------------------------------------------------------------------------------------
                                                    
    while(1):
        try:
            options_noise["decay_factor"] = float( input("Enter a float value(less than equal to 1), for DECAY FACTOR:" ) )
            if( options_noise["decay_factor"] > 1 ):
                raise AttributeError
        except AttributeError:
            print("DECAY FACTOR value cannot be greater than 1.\n Try again with valid values...")
            continue
        except ValueError:
            print("Please enter integer or float values only, in correct format...\nTry again, entering the values.")
            continue
        else:
            break
                                                    
    # -----------------------------------------------------------------------------------------------------------------------------------------
    
          
            

In [7]:
n=12 #MAX for dm dim

'''
Circuit generation  using 12 qubits max for dm simulator
'''
qubits=[*range(0,n)]
qreg_q = QuantumRegister(n, 'q')
creg_c = ClassicalRegister(n, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)


state = int(input(print("Mention the bell states you wanted to prepare and enter corresponding value the\n 1.(|00> + |11>)/root(2) \n 2.(|00> - |11>)/root(2) \n 3.(|01> + |10>)/root(2) \n 4.(|01> - |10>)/root(2)")))


Mention the bell states you wanted to prepare and enter corresponding value the
 1.(|00> + |11>)/root(2) 
 2.(|00> - |11>)/root(2) 
 3.(|01> + |10>)/root(2) 
 4.(|01> - |10>)/root(2)


None 4


In [8]:
'''
Generation of entangled pairs-6 pairs for 12 qubits
'''
#Generation of entangled pair 
if (state == 1):
    for i in range(0,12,2):
        circuit.h(i)
        circuit.cx(i,i+1)

if(state == 2):
    for i in range(0,12,2):
        circuit.x(i)
        circuit.h(i)
        circuit.cx(i,i+1)
if(state == 3):
    for i in range(0,12,2):
        circuit.x(i+1)
        circuit.h(i)
        circuit.cx(i,i+1)
if(state == 4):
    for i in range(0,12,2):
        circuit.x(i)
        circuit.x(i+1)
        circuit.h(i)
        circuit.cx(i,i+1)

circuit.barrier(qubits)

print("The basis encoded had done in the following form 0,1,2 denotes three measurement basis for alice and bob in xy plane \n Alice Basis: 0 denotes phi_1 = 0, 1 denotes phi_2 = pi/4 and 2 denotes phi_3 = pi/2\n Bob Basis: 0 denotes phi_1 = pi/4, 1 denotes phi_2 = pi/2 and 2 denotes phi_3 = 3*pi/4")


alice_bases=randint(3,size=6)
print("Alice choose the following basis ")
print(alice_bases)

#Sender Measurment
for i in range(6):
        if(alice_bases[i]==0):
            
            circuit.measure(i*2,i*2,basis="Ensemble",add_param="X")#0 degree 
        elif(alice_bases[i]==1):
            
           circuit.measure(i*2,i*2,basis="Ensemble",add_param=np.array(["N",np.array([0.707106,0.70710,0])]))#45 degree  
        else:
            circuit.measure(i*2,i*2, basis="Ensemble",add_param="Y")#90 
circuit.barrier(qubits)

bob_bases=randint(3,size=6)
print("Alice choose the following basis ")
print(bob_bases)
#Receiver Measurment
for i in range(6):
        if(bob_bases[i]==0):
            
            circuit.measure(i,i,basis="Ensemble",add_param=np.array(["N",np.array([0.707106,0.70710,0])]))#45 degree
        elif(bob_bases[i]==1):
            
            circuit.measure(i,i,basis="Ensemble",add_param="Y")#90 
        else:
            circuit.measure(i,i, basis="Ensemble",add_param=np.array(["N",np.array([-0.707106,0.70710,0])]))#135 degree
circuit.barrier(qubits)
circuit.draw()

The basis encoded had done in the following form 0,1,2 denotes three measurement basis for alice and bob in xy plane 
 Alice Basis: 0 denotes phi_1 = 0, 1 denotes phi_2 = pi/4 and 2 denotes phi_3 = pi/2
 Bob Basis: 0 denotes phi_1 = pi/4, 1 denotes phi_2 = pi/2 and 2 denotes phi_3 = 3*pi/4
Alice choose the following basis 
[1 1 2 2 1 0]


┌───┐┌───┐      ░ ┌─┐                ░ 
 q_0: ┤ X ├┤ H ├──■───░─┤M├────────────────░─
      ├───┤└───┘┌─┴─┐ ░ └╥┘                ░ 
 q_1: ┤ X ├─────┤ X ├─░──╫─────────────────░─
      ├───┤┌───┐└───┘ ░  ║ ┌─┐             ░ 
 q_2: ┤ X ├┤ H ├──■───░──╫─┤M├─────────────░─
      ├───┤└───┘┌─┴─┐ ░  ║ └╥┘             ░ 
 q_3: ┤ X ├─────┤ X ├─░──╫──╫──────────────░─
      ├───┤┌───┐└───┘ ░  ║  ║ ┌─┐          ░ 
 q_4: ┤ X ├┤ H ├──■───░──╫──╫─┤M├──────────░─
      ├───┤└───┘┌─┴─┐ ░  ║  ║ └╥┘          ░ 
 q_5: ┤ X ├─────┤ X ├─░──╫──╫──╫───────────░─
      ├───┤┌───┐└───┘ ░  ║  ║  ║ ┌─┐       ░ 
 q_6: ┤ X ├┤ H ├──■───░──╫──╫──╫─┤M├───────░─
      ├───┤└───┘┌─┴─┐ ░  ║  ║  ║ └╥┘       ░ 
 q_7: ┤ X ├─────┤ X ├─░──╫──╫──╫──╫────────░─
      ├───┤┌───┐└───┘ ░  ║  ║  ║  ║ ┌─┐    ░ 
 q_8: ┤ X ├┤ H ├──■───░──╫──╫──╫──╫─┤M├────░─
      ├───┤└───┘┌─┴─┐ ░  ║  ║  ║  ║ └╥┘    ░ 
 q_9: ┤ X ├─────┤ X ├─░──╫──╫──╫──╫──╫─────░─
      ├───┤┌───┐└───┘ ░  ║  ║  ║  ║  ║ ┌─┐ ░ 
q_10: ┤ X ├┤ H ├──■───░──╫──╫──╫──╫──╫─┤M├─░─
      ├───┤└───┘┌─┴─┐ ░  ║  ║  ║  ║  ║ └╥┘ ░ 
q_11: ┤ X ├─────┤ X ├─░──╫──╫──╫──╫──╫──╫──░─
      └───┘     └───┘ ░  ║  ║  ║  ║  ║  ║  ░ 
c: 12/═══════════════════╩══╩══╩══╩══╩══╩════
                         0  2  4  6  8  10

In [56]:


'''
Eve's dropping by EVE by measurement in his/her own choosen basis
'''
if("trace_pass" == "yes"):
    attacker_basis=randint(2,size=n)
    print("Attacker choose folowing basis" )
    print(attacker_basis)

    #Attacker Basis and Meaurement
    for i in range(n):
        if(attacker_basis[i]==0):
            circuit.measure(qreg_q[i], creg_c[i],basis="Ensemble",add_param="X")
        else:
            circuit.measure(qreg_q[i], creg_c[i],basis="Ensemble",add_param="Z")

'''
Measurment by BOB in his own choosen basis
'''

reciever_basis=randint(2,size=n)
print("Reciever choose folowing basis" )
print(reciever_basis)

#Reciever Basis and Meaurement
for i in range(n):
    if(reciever_basis[i]==0):
        circuit.measure(qreg_q[i], creg_c[i],basis="Ensemble",add_param="X")
    else:
        circuit.measure(qreg_q[i], creg_c[i],basis="Ensemble",add_param="Z")

'''
The experimental run of the protocol in presence of 2016 qubits

'''
if("trace_pass" == "yes"):
    s_key,r_key=enhanced_key_eve(n,envs)
else:
    s_key,r_key=enhanced_key(n,envs)
    

print("These are the common key shared")
print(s_key)
print(r_key)

s=math.ceil(len(s_key) * 0.5)
print(" Sample .... ", s)
sample_size=randint(0,s)

print("length of sifted key is ",len(s_key) )
print("length of sample taken is ",sample_size)
sample_string = randint(n, size=sample_size)

sender_sample=sampling(s_key,sample_string)
reciever_sample=sampling(r_key,sample_string)

mismatch = 0
for i in range(sample_size):
    if(sender_sample[i]!=reciever_sample[i]):
        mismatch = mismatch+1

print("Sender_Sample")
print(sender_sample)
print("Receiver_Sample")
print(reciever_sample)

error_rate = 0
if sender_sample==reciever_sample:
    print("Cryptographic_key_is")
    print(s_key)
    error_rate = 0
else:
    print("error or Eve's Dropping")
    error_rate=mismatch/sample_size
    print("Error_rate=",error_rate)

'''
Secret Key Rate 
'''
if(error_rate!=0):
    secret_key_rate = skr(len(s_key),error_rate)

print("Secret Key Rate for simulation in non-ideal enviornment is"+ secret_key_rate) 

[[0.25+0.j 0.  +0.j 0.  +0.j 0.25+0.j]
 [0.  +0.j 0.25+0.j 0.25+0.j 0.  +0.j]
 [0.  +0.j 0.25+0.j 0.25+0.j 0.  +0.j]
 [0.25+0.j 0.  +0.j 0.  +0.j 0.25+0.j]]
